<a href="https://colab.research.google.com/github/deepesh321/Facial_recognition/blob/master/facial_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import re
import numpy as np
from PIL import Image
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Input, Lambda, Dense, Dropout, Convolution2D, MaxPooling2D, Flatten,BatchNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import VGG16
from tensorflow.keras.activations import relu,tanh

%tensorflow_version 1.x

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
anchor=[]
pos_neg=[]
y_values=[0]*226
count=0
BASE_DIR = '/content/drive/My Drive/facial_recog'
for i in range(113):
    #for i in range(int(565/113)):
    ind1=0
    ind2=0
    while ind1==ind2:
        ind1 = np.random.randint(10)
        ind2 = np.random.randint(10)
    img1 = image.load_img(BASE_DIR + '/male/' + 's'+ str(i) + '/' + str(ind1 + 1) + '.jpg',target_size=(112,112))
    img2 = image.load_img(BASE_DIR+'/male/' +'s'+ str(i) + '/' + str(ind2 + 1) + '.jpg',target_size=(112,112))
    img1 = image.img_to_array(img1)
    img2 = image.img_to_array(img2)
    img1 = np.expand_dims(img1, axis=0)
    img2 = np.expand_dims(img2, axis=0)
    anchor.append(img1)
    pos_neg.append(img2)
    y_values[count]=1
    count=count+1

for i in range(113):
    ind1=0
    while ind1==i:
        ind1 = np.random.randint(113)
    a=np.random.randint(20)
    img1 = image.load_img(BASE_DIR + '/male/' + 's'+ str(i) + '/' + str(a + 1) + '.jpg',target_size=(112,112))
    img2 = image.load_img(BASE_DIR+'/male/' +'s' + str(ind1) + '/' + str(a + 1) + '.jpg',target_size=(112,112))
    img1 = image.img_to_array(img1)
    img2 = image.img_to_array(img2)
    img1 = np.expand_dims(img1, axis=0)
    img2 = np.expand_dims(img2, axis=0)
    anchor.append(img1)
    pos_neg.append(img2)
    y_values[count]=0
    count=count+1



In [16]:
anchor=np.array(anchor)
pos_neg=np.array(pos_neg)
anchor = np.rollaxis(anchor, 1, 0)
anchor = anchor[0]
pos_neg= np.rollaxis(pos_neg, 1, 0)
pos_neg = pos_neg[0]
y_values=np.array(y_values)
y_values=y_values.reshape(y_values.shape[0],1)
print(anchor.shape,pos_neg.shape,y_values.shape)

(226, 112, 112, 3) (226, 112, 112, 3) (226, 1)


In [0]:
def build_model():
    #input_image=Input(shape=(224,224,3))
    model=Sequential()
    model.add(VGG16(weights=None,include_top=True,input_shape=(112,112,3)))
    model.layers[-1].activation = tanh
    model.add(Dropout(0.25))
    model.add(Dense(128))
    #out = Dense(128)(model.layers[-1].output)
    return model

In [0]:
def triplet_loss(vects):
    anchor,positive=vects
    print(anchor.shape,positive.shape)
    loss = tf.reduce_sum((anchor - positive)**2, axis=-1)
    print(loss)
    print(loss.shape)    
    return loss

def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

In [0]:
def contrastive_loss(y_true, y_pred):
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

In [0]:
image_shape=(112,112,3)
def get_model():
    anchor_input=Input(shape=image_shape)
    pos_neg_input=Input(shape=image_shape)
    my_model=build_model()

    pos_neg_encoding=my_model(pos_neg_input)
    anchor_encoding=my_model(anchor_input)
    distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([anchor_encoding,pos_neg_encoding])
    # distance=Activation('sigmoid')
    return Model(inputs=[anchor_input,pos_neg_input],outputs=distance)

In [47]:
fr_model=get_model()
fr_model.compile(loss=contrastive_loss, optimizer='adam', metrics=['mae'])
fr_model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           [(None, 112, 112, 3) 0                                            
__________________________________________________________________________________________________
input_22 (InputLayer)           [(None, 112, 112, 3) 0                                            
__________________________________________________________________________________________________
sequential_7 (Sequential)       (None, 128)          54599592    input_22[0][0]                   
                                                                 input_21[0][0]                   
__________________________________________________________________________________________________
lambda_5 (Lambda)               (None, 1)            0           sequential_7[2][0]         

In [48]:
fr_model.fit([anchor/255.0,pos_neg/255.0],y_values,batch_size=64,epochs=10)

Train on 226 samples
Epoch 1/10
226/226 [==============================] - 17s 77ms/sample - loss: 0.4879 - mean_absolute_error: 0.5000
Epoch 2/10
226/226 [==============================] - 17s 74ms/sample - loss: 0.4875 - mean_absolute_error: 0.5001
Epoch 3/10
226/226 [==============================] - 17s 75ms/sample - loss: 0.4874 - mean_absolute_error: 0.5000
Epoch 4/10
226/226 [==============================] - 17s 74ms/sample - loss: 0.4874 - mean_absolute_error: 0.5000
Epoch 5/10
226/226 [==============================] - 17s 74ms/sample - loss: 0.4871 - mean_absolute_error: 0.4999
Epoch 6/10
226/226 [==============================] - 16s 73ms/sample - loss: nan - mean_absolute_error: nan
Epoch 7/10
 64/226 [=======>......................] - ETA: 11s - loss: nan - mean_absolute_error: nan

KeyboardInterrupt: ignored